In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

files = os.listdir('data_preprocessed_python\\')

list_of_dfs = []
for file in files:
    with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
        
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        allData = allData.merge(aux, left_index=True, right_index=True)
        
        list_of_dfs.append(allData)
big_df = pd.concat(list_of_dfs, ignore_index=True)#ignore_index to reset index of big_df


In [1]:
data = big_df.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] <5, 'happy', 'sad')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

data


NameError: name 'big_df' is not defined

In [11]:
from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

<ipython-input-12-353746cc4ea1>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  4.8min
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed: 11.0min finished


RandomForestClassifier(max_depth=2, n_jobs=5, random_state=0, verbose=1)

In [13]:
predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

accuracy_score(yTest, predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    1.7s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    4.3s finished


0.650055900452629